# Automatic generation of coupled cluster equations

## Load wick&d

In [25]:
import wicked as w
import time
from IPython.display import display, Math, Latex

def latex(expr):
    """Function to render any object that has a member latex() function"""
    display(Math(expr.latex()))

# Define the orbital spaces
Here we define two orbital space and assign indices
1. Occupied
2. Virtuals

In [19]:
w.reset_space()
w.add_space('v','fermion','unoccupied',['a','b','c','d','e','f'])
w.add_space('o','fermion','occupied',['i','j','k','l','m','n'])

## Define the Hamiltonian operator

In [20]:
E0 = w.op("E_0",[""])
F = w.utils.gen_op('f',1,'ov','ov')
V = w.utils.gen_op('v',2,'ov','ov')
H = E0 + F + V

## Define a function to compute the cluster operator truncated to level *n*

In [21]:
def make_T(n):
    components = []
    for k in range(1,n + 1):
        label = f"{'v+' * k} {'o' * k}"
        components.append(label)
    return w.op("t",components)

## Setup the similarity-transformed Hamiltonian and compute expectation value

In [38]:
def cc_equations(n):
    start = time.perf_counter()
    wt = w.WickTheorem()

    T = make_T(n)
    Hbar = w.bch_series(H,T,4)
    expr = wt.contract(w.rational(1), Hbar, 0, 2 * n)
    mbeq = expr.to_manybody_equation("r")
    end = time.perf_counter()    
    t = end - start
    print(f'Generated equations in {t} seconds')

    equations = {}
    for r in range(1,n + 1):
        s = f"{'o' * r}|{'v' * r}" 
        equations[r] = (mbeq[s])
        
    for rank,eq in equations.items():
        print(f'Rank {rank}: {len(eq)} equations')        
        
    return equations

equations = cc_equations(4)

Generated equations in 2.439585500000021 seconds
Rank 1: 15 equations
Rank 2: 38 equations
Rank 3: 53 equations
Rank 4: 74 equations
